In [1]:
import comet
import hydra 
from omegaconf import DictConfig, OmegaConf
import pyrootutils
from src import utils
import os
from pytorch_lightning import seed_everything
from src.utils.resolve import set_env
from src.utils import instantiator as inst
from src.utils import setting_environment
from src.utils import task_wrapper


root = pyrootutils.setup_root(
    search_from=os.getcwd(),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)
_HYDRA_PARAMS = {
    "version_base": "1.3",
    "config_path": str(root / "configs"),
    "config_name": "train.yamls",
}


In [2]:
import os
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from omegaconf import OmegaConf

GlobalHydra.instance().clear()

hydra.initialize(config_path="../configs")
cfg = hydra.compose(config_name="train", return_hydra_config=True)
HydraConfig().cfg = cfg
setting_environment(cfg.env)
OmegaConf.resolve(cfg)


/tmp/ipykernel_478853/3531891309.py:10: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path="../configs")


In [3]:
datamodule= hydra.utils.instantiate(cfg.datamodule,_recursive_=False)
datamodule.setup()

In [4]:
datamodule_vis = hydra.utils.instantiate(cfg.visualisation,datamodule=datamodule)
# datamodule_vis.visualise()

<function plot_psd_fn at 0x7febb7545cf0>
<function plot_psd_fn.<locals>.wrap at 0x7febb7308af0>


In [5]:
callbacks = inst.instantiate_callbacks(cfg.callbacks)

In [6]:
logger = inst.instantiate_loggers(cfg.logger) 

CometLogger will be initialized in online mode


In [7]:
modelmodule = hydra.utils.instantiate(cfg.backbone)

1537


In [8]:
trainer = hydra.utils.instantiate(cfg.trainer,callbacks=callbacks,logger=logger)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model=modelmodule,datamodule=datamodule)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: You are trying to log string value as a metric. This is not recommended.
COMET INFO: Experiment is live on comet.com https://www.comet.com/yacinebelhadj/ssl/1757402a57cf45a1bb9b2c3fbda4d3ea

/home/yacine/miniconda3/envs/ssl_dd/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /home/yacine/Documents/PhD/Code/Package/ssl_dd/src/None/checkpoints exists and is not empty.


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name  ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model │ SimpleDenseNet   │  1.1 M │
│ 1 │ loss  │ CrossEntropyLoss │      0 │
└───┴───────┴──────────────────┴────────┘

Trainable params: 1.1 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.1 M                                                                                                
Total estimated model params size (MB): 4

Output()

`Trainer.fit` stopped: `max_epochs=2` reached.


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/yacinebelhadj/ssl/1757402a57cf45a1bb9b2c3fbda4d3ea
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr-Adam            : 0.0001
COMET INFO:     train_Accuracy [2] : (0.18326270580291748, 0.4338858425617218)
COMET INFO:     train_loss [2]     : (2.2078380584716797, 2.718574285507202)
COMET INFO:     val_Accuracy [2]   : (0.8197906017303467, 0.9184945225715637)
COMET INFO:     val_loss [2]       : (1.748160719871521, 2.1952545642852783)
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:   Parameters:
COMET INFO:     cfg/logging/on_epoch                : True
COMET INFO:     cfg/logging/on_s

In [10]:
# get the best model path 
best_model_path = trainer.checkpoint_callback.best_model_path

In [11]:
cfg.backbone._target_

'src.model.classifier.ClassifierModule'

In [12]:
d= hydra.utils.instantiate({'_target_':cfg.backbone._target_},_partial_=True)
d

functools.partial(<class 'src.model.classifier.ClassifierModule'>)

In [13]:
 modelmodule
# let's get an instance of the model and not instantiate it again 

ClassifierModule(
  (model): SimpleDenseNet(
    (layers): ModuleList(
      (0): Linear(in_features=1537, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.5, inplace=False)
      (8): Linear(in_features=512, out_features=128, bias=True)
      (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): Dropout(p=0.5, inplace=False)
      (12): Linear(in_features=128, out_features=32, bias=True)
      (13): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): Dropout(p=0.5, inplace=False)
    )
    (encoder): Sequential(
      (0): Linear(in_features=1

In [29]:
model.load_from_checkpoint(best_model_path)

1537


ClassifierModule(
  (model): SimpleDenseNet(
    (layers): ModuleList(
      (0): Linear(in_features=1537, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.5, inplace=False)
      (8): Linear(in_features=512, out_features=128, bias=True)
      (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): Dropout(p=0.5, inplace=False)
      (12): Linear(in_features=128, out_features=32, bias=True)
      (13): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): Dropout(p=0.5, inplace=False)
    )
    (encoder): Sequential(
      (0): Linear(in_features=1

In [28]:
import importlib

print(cfg.backbone._target_)
module = importlib.import_module('.'.join(cfg.backbone._target_.split('.')[:-1]))
model_name = cfg.backbone._target_.split('.')[-1]

model = getattr(module,model_name)

src.model.classifier.ClassifierModule


In [33]:
load_object(cfg.backbone._target_).load_from_checkpoint(best_model_path)

1537


ClassifierModule(
  (model): SimpleDenseNet(
    (layers): ModuleList(
      (0): Linear(in_features=1537, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.5, inplace=False)
      (8): Linear(in_features=512, out_features=128, bias=True)
      (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): Dropout(p=0.5, inplace=False)
      (12): Linear(in_features=128, out_features=32, bias=True)
      (13): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): Dropout(p=0.5, inplace=False)
    )
    (encoder): Sequential(
      (0): Linear(in_features=1

In [30]:
import importlib

print(cfg.backbone._target_)
model = importlib.import_module(cfg.backbone._target_)


src.model.classifier.ClassifierModule


ModuleNotFoundError: No module named 'src.model.classifier.ClassifierModule'; 'src.model.classifier' is not a package

In [ ]:
trainer.test(model=modelmodule,datamodule=datamodule)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/yacinebelhadj/ssl/e18c3e65a1be4b168aacb0765f4e72cb



Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_Accuracy       │    0.9524999856948853     │
│         test_loss         │    1.7757247686386108     │
└───────────────────────────┴───────────────────────────┘

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/yacinebelhadj/ssl/e18c3e65a1be4b168aacb0765f4e72cb
COMET INFO:   Metrics:
COMET INFO:     test_Accuracy : 0.9524999856948853
COMET INFO:     test_loss     : 1.7757247686386108
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:   Parameters:
COMET INFO:     cfg/logging/on_epoch                : True
COMET INFO:     cfg/logging/on_step                 : False
COMET INFO:     cfg/logging/prog_bar                : True
COMET INFO:     cfg/logging/sync_dist               : True
COMET INFO:     cfg/loss/_target_                   : torch.nn.CrossEntropyLoss
COMET INFO:     cfg/metric/_target_   

[{'test_loss': 1.7757247686386108, 'test_Accuracy': 0.9524999856948853}]

In [ ]:
dds = hydra.utils.instantiate(cfg.downstream,encoder=modelmodule.model.encoder)
dds.fit(datamodule.train_dataloader())

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
evaluator = hydra.utils.instantiate(cfg.eval,dds=dds,datamodule=datamodule,logger=logger)

In [ ]:
evaluator.setup()

Output()

Output()

Output()

Output()

In [ ]:
evaluator.evaluate_vas()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


[{'system_name': 0, 'auc': 0.8454100921075346, 'individual_auc': [{'f_anomaly': 1.0, 'a_anomaly': -0.2, 'auc': 0.92955, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': -0.15, 'auc': 0.905925, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': -0.1, 'auc': 0.8748749999999998, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': -0.05, 'auc': 0.7002999999999999, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': 0.0, 'auc': 0.50015, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': 0.05, 'auc': 0.690825, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': 0.1, 'auc': 0.861175, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': 0.15, 'auc': 0.9061499999999999, 'saturation_applied': 0}, {'f_anomaly': 1.0, 'a_anomaly': 0.2, 'auc': 0.933825, 'saturation_applied': 0}, {'f_anomaly': 2.0, 'a_anomaly': -0.2, 'auc': 0.9783000000000001, 'saturation_applied': 0}, {'f_anomaly': 2.0, 'a_anomaly': -0.15, 'auc': 0.916125, 'saturation_applied': 0}, {'f_a

COMET INFO: Experiment is live on comet.com https://www.comet.com/yacinebelhadj/ssl/a4bdb81f971c46d1b7b1579c34d6b0e6



TypeError: list indices must be integers or slices, not list

In [ ]:
vas, all_auc= evaluator.benchmark_vas.evaluate_all()

In [ ]:
import pandas as pd 
from src.eval.utils_plot import plot_heatmap
df= pd.DataFrame(all_auc)
# iterate over each row 
for _,rows in df.iterrows(): 
    title_name = f"System {rows.system_name}: AUC = {rows.auc:.3f}" 
    data = pd.DataFrame(rows.individual_auc)[['f_anomaly','a_anomaly','auc']]
    fig,ax=plot_heatmap(data,axis_label=['f_anomaly','a_anomaly'],title=title_name)
    

In [ ]:
#plot the figure 
score_sa, df = evaluator.benchmark_sa.evaluate_all()

In [ ]:
hue = evaluator.benchmark_sa.hue

In [ ]:
from src.eval.utils_plot import plot_boxplot


In [ ]:
from typing import List

In [ ]:
def plot_boxplot(data,axis:List[str],title:str,ax=None):
    if ax is None:
        fig,ax = plt.subplots()
 

In [ ]:
df_plot= evaluator.benchmark_sa.combined_df
df_grouped = df_plot.groupby(hue)
for name, group in df_grouped:
    fig,ax=plot_boxplot(group,axis_label=['f_anomaly','a_anomaly'],title=name)

KeyError: 'state'

In [ ]:
_,df  = evaluator.benchmark_sa.evaluate_all()

In [ ]:
df = pd.DataFrame(df)
df.rename(columns={'auc':'global_auc'},inplace=True)
df_exploded = df.explode('individual_auc').reset_index(drop=True)
normalized_data = pd.json_normalize(df_exploded['individual_auc'])
df_result = pd.concat([df_exploded, normalized_data], axis=1)
df_result.drop('individual_auc', axis=1, inplace=True)
df_result = df_result[df_result['stiffness_reduction']==0.03][['system_name','auc']]
df_result.set_index('system_name',inplace=True)
dict_auc_003= df_result.to_dict()['auc']
dict_auc_003 = {f"System {key}":value for key,value in dict_auc_003.items()}


In [ ]:
# add to key the string system name


{0: 0.6208250000000001,
 1: 0.7310000000000001,
 2: 0.5518000000000001,
 3: 0.6188,
 4: 0.52305,
 5: 0.7492000000000001,
 6: 0.5566250000000001,
 7: 0.62995,
 8: 0.60955,
 9: 0.5501,
 10: 0.5823,
 11: 0.6798750000000001,
 12: 0.637375,
 13: 0.58605,
 14: 0.569375,
 15: 0.815825,
 16: 0.512125,
 17: 0.705625,
 18: 0.60815,
 19: 0.5914499999999999}

{0: 0.6208250000000001,
 1: 0.7310000000000001,
 2: 0.5518000000000001,
 3: 0.6188,
 4: 0.52305,
 5: 0.7492000000000001,
 6: 0.5566250000000001,
 7: 0.62995,
 8: 0.60955,
 9: 0.5501,
 10: 0.5823,
 11: 0.6798750000000001,
 12: 0.637375,
 13: 0.58605,
 14: 0.569375,
 15: 0.815825,
 16: 0.512125,
 17: 0.705625,
 18: 0.60815,
 19: 0.5914499999999999}

TypeError: DataFrame.explode() missing 1 required positional argument: 'column'

In [ ]:
# from the list df   [{'system_name': 0, 'auc': 0.7256412865975094,'individual_auc': [{'stiffness_reduction': 0.01,  'auc': 0.4954 'saturation_applied': 0},  {'stiffness_reduction': 0.03,   'auc': 0.6208250000000001,
# get the individual auc with stiffness_reduction = 0.03 with conditionning operation 
df['individual_auc'].explode().apply(pd.Series).reset_index()


,index,stiffness_reduction,auc,saturation_applied
0,0,0.01,0.495400,0.0
1,0,0.03,0.620825,0.0
2,0,0.05,0.704875,0.0
3,0,0.07,0.795725,0.0
4,0,0.09,0.876625,0.0
...,...,...,...,...
135,19,0.05,0.727025,0.0
136,19,0.07,0.793850,0.0
137,19,0.09,0.850575,0.0
138,19,0.11,0.902350,0.0


In [ ]:
group.loc[group['condition']=='before', "stiffness_reduction"] = 0


In [ ]:
group['anomaly_index'] = - group['anomaly_index']

In [ ]:
group['anomaly_index']  = group['anomaly_index'] - group['anomaly_index'].min() + 1

In [ ]:
import matplotlib.pyplot as plt
group.boxplot(column='anomaly_index', by='stiffness_reduction',showfliers=False)
plt.yscale('log')
plt.show()